In [1]:
from recordclass import dataobject, __version__
from math import sqrt
import gc
import sys

In [2]:
print(sys.version)
print(__version__)

3.13.2 (main, Feb  5 2025, 01:23:35) [GCC 14.2.0]
0.22.1


In [4]:
class Point(dataobject):
    x:int
    y:int

class Point3(Point):
    z:int

class Vector(Point):
    #
    def sizeof(self):
        return self.__sizeof__()

class Vector3(Point3):
    #
    def sizeof(self):
        return self.__sizeof__()


In [13]:
def test_point(N):
    L = []
    for i in range(N):
        L.append(Point(1,2))
    return L

def test_point3(N):
    L = []
    for i in range(N):
        L.append(Point3(1,2,3))
    return L

def test_vector(N):
    L = []
    for i in range(N):
        L.append(Point(1,2))
    return L

def test_vector3(N):
    L = []
    for i in range(N):
        L.append(Vector3(1,2,3))
    return L 

def test_vector_sizeof(N):
    v = Vector(1,2)
    L = []
    for i in range(N):
        L.append(v.sizeof())
    return L

def test_vector3_sizeof(N):
    v = Vector3(1,2,3)
    L = []
    for i in range(N):
        L.append(v.sizeof())
    return L


In [5]:
print("Sizeof:")
print("Point:", sys.getsizeof(Point(1,2)))
print("Vector:", sys.getsizeof(Vector(1,2)))
print("Point3:", sys.getsizeof(Point3(1,2,3)))
print("Vector3:", sys.getsizeof(Vector3(1,2,3)))

Sizeof:
Point: 32
Vector: 32
Point3: 40
Vector3: 40


In [6]:
gc.collect()
%timeit test_point(1000)

64 μs ± 1.8 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
gc.collect()
%timeit test_point3(1000)

72.4 μs ± 10.4 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
gc.collect()
%timeit test_vector(1000)

64 μs ± 254 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
gc.collect()
%timeit test_vector3(1000)

69 μs ± 1.66 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
gc.collect()
%timeit test_vector_sizeof(1000)

73.7 μs ± 458 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
gc.collect()
%timeit test_vector3_sizeof(1000)

75.8 μs ± 1.4 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
